## Comparing Python vs TensorFlow Performance

To justify why we use TensorFlow over normal python, we can run some benchmarks for simple operations and compare the two implementations.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time as time

### Matrix Multiplication

We compare the time takes for Python and TensorFlow to evaluate the product of two matrices of varying sizes.

In [2]:
n_replicate = 15
matrix_sizes = [2,8,32,128,512,2048,8192,16384]

#### In Python

In [3]:
for n in matrix_sizes:
    run_time = []
    for i in range(n_replicate):
        start = time.time()
        a = np.random.uniform(size=(n,n))
        b = np.random.uniform(size=(n,n))
        c = np.matmul(a,b)
        end = time.time()
        run_time.append(end-start)
    mean = np.mean(run_time)
    sd = np.std(run_time)
    print("For a {}x{} Matrix, Runtime = {:0.4f} +/- {:0.4f} secs".format(n,n,mean,sd))

For a 2x2 Matrix, Runtime = 0.0179 +/- 0.0668 secs
For a 8x8 Matrix, Runtime = 0.0001 +/- 0.0001 secs
For a 32x32 Matrix, Runtime = 0.0001 +/- 0.0001 secs
For a 128x128 Matrix, Runtime = 0.0009 +/- 0.0005 secs
For a 512x512 Matrix, Runtime = 0.0110 +/- 0.0013 secs
For a 2048x2048 Matrix, Runtime = 0.2739 +/- 0.0167 secs
For a 8192x8192 Matrix, Runtime = 7.9106 +/- 0.3574 secs
For a 16384x16384 Matrix, Runtime = 51.0665 +/- 8.6048 secs


#### In TensorFlow

In [4]:
for n in matrix_sizes:
    run_time = []
    for i in range(n_replicate):
        start = time.time()
        a = tf.random_uniform([n,n])
        b = tf.random_uniform([n,n])
        c = tf.matmul(a,b)
        with tf.Session() as sess:
            c = sess.run(c)
        end = time.time()
        run_time.append(end-start)
    mean = np.mean(run_time)
    sd = np.std(run_time)
    print("For a {}x{} Matrix, Runtime = {:0.4f} +/- {:0.4f} secs".format(n,n,mean,sd))

For a 2x2 Matrix, Runtime = 0.0815 +/- 0.2284 secs
For a 8x8 Matrix, Runtime = 0.0173 +/- 0.0012 secs
For a 32x32 Matrix, Runtime = 0.0204 +/- 0.0010 secs
For a 128x128 Matrix, Runtime = 0.0253 +/- 0.0026 secs
For a 512x512 Matrix, Runtime = 0.0297 +/- 0.0009 secs
For a 2048x2048 Matrix, Runtime = 0.0783 +/- 0.0038 secs
For a 8192x8192 Matrix, Runtime = 2.2284 +/- 0.0444 secs
For a 16384x16384 Matrix, Runtime = 16.8448 +/- 0.1515 secs


### RK4 Integration

We compare the time takes for Python and TensorFlow to evaluate the integrate varying numbers of the differential equations of the form $\dot x = \sin{xt}$ .

In [5]:
n_replicate = 15
equation_sizes = [1,10,100,1000,10000,100000,1000000]
t = np.arange(0,5,0.01)

#### In Python

In [6]:
def python_check_type(y,t): # Ensure Input is Correct
    return y.dtype == np.floating and t.dtype == np.floating

class python_Integrator():
    
    def integrate(self,func,y0,t):
        time_delta_grid = t[1:] - t[:-1]
        
        y = np.zeros((y0.shape[0],t.shape[0]))
        y[:,0] = y0

        for i in range(time_delta_grid.shape[0]):
            k1 = func(y[:,i], t[i])                               # RK4 Integration Steps
            half_step = t[i] + time_delta_grid[i] / 2
            k2 = func(y[:,i] + time_delta_grid[i] * k1 / 2, half_step)
            k3 = func(y[:,i] + time_delta_grid[i] * k2 / 2, half_step)
            k4 = func(y[:,i] + time_delta_grid[i] * k3, t[i] + time_delta_grid[i])
            y[:,i+1]= (k1 + 2 * k2 + 2 * k3 + k4) * (time_delta_grid[i] / 6) + y[:,i]
        return y

def odeint_python(func,y0,t):
    y0 = np.array(y0)
    t = np.array(t)
    if python_check_type(y0,t):
        return python_Integrator().integrate(func,y0,t)
    else:
        print("error encountered")
        
def f(X,t):
    return np.sin(X*t)

for n in equation_sizes:
    run_time = []
    for i in range(n_replicate):
        start = time.time()
        solution = odeint_python(f,[0.]*n,t)
        end = time.time()
        run_time.append(end-start)
    mean = np.mean(run_time)
    sd = np.std(run_time)
    print("For a {} Equations, Runtime = {:0.4f} +/- {:0.4f} secs".format(n,mean,sd))

For a 1 Equations, Runtime = 0.0149 +/- 0.0011 secs
For a 10 Equations, Runtime = 0.0187 +/- 0.0024 secs
For a 100 Equations, Runtime = 0.0171 +/- 0.0008 secs
For a 1000 Equations, Runtime = 0.0443 +/- 0.0016 secs
For a 10000 Equations, Runtime = 0.3593 +/- 0.0172 secs
For a 100000 Equations, Runtime = 3.6114 +/- 0.0844 secs
For a 1000000 Equations, Runtime = 71.2494 +/- 3.3895 secs


#### In TensorFlow

In [7]:
def tf_check_type(t, y0): # Ensure Input is Correct
    if not (y0.dtype.is_floating and t.dtype.is_floating):
        raise TypeError('Error in Datatype')

class Tf_Integrator():
    
    def integrate(self, func, y0, t): 
        time_delta_grid = t[1:] - t[:-1]
        
        def scan_func(y, t_dt): 
            t, dt = t_dt
            dy = self._step_func(func,t,dt,y)
            return y + dy

        y = tf.scan(scan_func, (t[:-1], time_delta_grid),y0)
        return tf.concat([[y0], y], axis=0)
    
    def _step_func(self, func, t, dt, y):
        k1 = func(y, t)
        half_step = t + dt / 2
        dt_cast = tf.cast(dt, y.dtype) # Failsafe

        k2 = func(y + dt_cast * k1 / 2, half_step)
        k3 = func(y + dt_cast * k2 / 2, half_step)
        k4 = func(y + dt_cast * k3, t + dt)
        return tf.add_n([k1, 2 * k2, 2 * k3, k4]) * (dt_cast / 6)
    

def odeint_tf(func, y0, t):
    t = tf.convert_to_tensor(t, preferred_dtype=tf.float64, name='t')
    y0 = tf.convert_to_tensor(y0, name='y0')
    tf_check_type(y0,t)
    return Tf_Integrator().integrate(func,y0,t)
        
def f(X,t):
    return tf.sin(X*t)

for n in equation_sizes:
    run_time = []
    for i in range(n_replicate):
        start = time.time()
        state = odeint_tf(f,tf.constant([0.]*n,dtype=tf.float64),t)
        with tf.Session() as sess:
            state = sess.run(state)
        end = time.time()
        run_time.append(end-start)
    mean = np.mean(run_time)
    sd = np.std(run_time)
    print("For a {} Equations, Runtime = {:0.4f} +/- {:0.4f} secs".format(n,mean,sd))

For a 1 Equations, Runtime = 0.2293 +/- 0.0860 secs
For a 10 Equations, Runtime = 0.2520 +/- 0.0319 secs
For a 100 Equations, Runtime = 0.2899 +/- 0.0337 secs
For a 1000 Equations, Runtime = 0.3456 +/- 0.0422 secs
For a 10000 Equations, Runtime = 0.6007 +/- 0.0233 secs
For a 100000 Equations, Runtime = 1.8797 +/- 0.0742 secs
For a 1000000 Equations, Runtime = 9.8849 +/- 0.0891 secs
